In [1]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -q tiny-imagenet-200.zip

--2025-01-28 14:33:22--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.64.64
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs231n.stanford.edu/tiny-imagenet-200.zip [following]
--2025-01-28 14:33:22--  https://cs231n.stanford.edu/tiny-imagenet-200.zip
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  49.9MB/s    in 5.0s    

2025-01-28 14:33:27 (46.9 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [2]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import os, glob
from torchvision.io import read_image, ImageReadMode
from PIL import Image

batch_size = 64

id_dict = {}
for i, line in enumerate(open('/content/tiny-imagenet-200/wnids.txt', 'r')):
  id_dict[line.replace('\n', '')] = i

class TrainTinyImageNetDataset(Dataset):
    def __init__(self, id, transform=None):
        self.filenames = glob.glob("/content/tiny-imagenet-200/train/*/*/*.JPEG")
        self.transform = transform
        self.id_dict = id

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = read_image(img_path)
        if image.shape[0] == 1:
          image = read_image(img_path,ImageReadMode.RGB)
        label = self.id_dict[img_path.split('/')[4]]
        if self.transform:
            image = self.transform(image.type(torch.FloatTensor))
        return image, label

class TestTinyImageNetDataset(Dataset):
    def __init__(self, id, transform=None):
        self.filenames = glob.glob("/content/tiny-imagenet-200/val/images/*.JPEG")
        self.transform = transform
        self.id_dict = id
        self.cls_dic = {}
        for i, line in enumerate(open('/content/tiny-imagenet-200/val/val_annotations.txt', 'r')):
            a = line.split('\t')
            img, cls_id = a[0],a[1]
            self.cls_dic[img] = self.id_dict[cls_id]


    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = read_image(img_path)
        if image.shape[0] == 1:
          image = read_image(img_path,ImageReadMode.RGB)
        label = self.cls_dic[img_path.split('/')[-1]]
        if self.transform:
            image = self.transform(image.type(torch.FloatTensor))
        return image, label

class RealTestTinyImageNetDataset(Dataset):
    def __init__(self, id, transform=None):
        self.filenames = glob.glob("/content/tiny-imagenet-200/test/images/*.JPEG")  # Ubah path ke folder test
        self.transform = transform
        self.id_dict = id

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = read_image(img_path)
        if image.shape[0] == 1:
            image = read_image(img_path, ImageReadMode.RGB)
        # Karena tidak ada label di folder test, kita bisa mengembalikan -1 atau None
        label = -1  # Atau None jika tidak ada label
        if self.transform:
            image = self.transform(image.type(torch.FloatTensor))
        return image, label

# Hapus transformasi normalisasi dan ganti dengan transformasi resize
transform = transforms.Compose([
    transforms.Resize((224, 224)),
])

# Fungsi untuk resize dan timpa gambar
def resize_and_overwrite_images(dataset):
    for img_path in dataset.filenames:
        image = Image.open(img_path)
        image = transform(image)
        image.save(img_path)

# Resize dan timpa gambar pada dataset train
trainset = TrainTinyImageNetDataset(id=id_dict)
resize_and_overwrite_images(trainset)

# Resize dan timpa gambar pada dataset test
testset = TestTinyImageNetDataset(id=id_dict)
resize_and_overwrite_images(testset)

# Setelah resize, Anda dapat membuat dataset baru dengan transformasi yang diinginkan
trainset = TrainTinyImageNetDataset(id=id_dict, transform=transforms.ToTensor())
testset = TestTinyImageNetDataset(id=id_dict, transform=transforms.ToTensor())

In [4]:

class RealTestTinyImageNetDataset(Dataset):
    def __init__(self, id, transform=None):
        self.filenames = glob.glob("/content/tiny-imagenet-200/test/images/*.JPEG")  # Ubah path ke folder test
        self.transform = transform
        self.id_dict = id

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_path = self.filenames[idx]
        image = read_image(img_path)
        if image.shape[0] == 1:
            image = read_image(img_path, ImageReadMode.RGB)
        # Karena tidak ada label di folder test, kita bisa mengembalikan -1 atau None
        label = -1  # Atau None jika tidak ada label
        if self.transform:
            image = self.transform(image.type(torch.FloatTensor))
        return image, label

# Resize dan timpa gambar pada dataset test
realtestset = RealTestTinyImageNetDataset(id=id_dict)
resize_and_overwrite_images(realtestset)


In [5]:
import shutil

# Path folder yang akan di-zip
folder_to_zip = '/content/tiny-imagenet-200'

# Nama file zip yang akan dibuat
zip_filename = '/content/jawaterbaru.zip'

# Membuat file zip
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', folder_to_zip)

print(f"Folder {folder_to_zip} telah di-zip dan disimpan sebagai {zip_filename}")

Folder /content/tiny-imagenet-200 telah di-zip dan disimpan sebagai /content/jawaterbaru.zip
